In [ ]:
from tensorflow import keras

In [ ]:
IMG_SIZE = (150, 150)

train_ds = keras.utils.image_dataset_from_directory(
    'data/cats_dogs/',
    batch_size=16,
    image_size=IMG_SIZE,
    shuffle=True,
    seed=42,
    validation_split=0.3,
    subset='training')

val_ds = keras.utils.image_dataset_from_directory(
    'data/cats_dogs/',
    batch_size=16,
    image_size=IMG_SIZE,
    shuffle=True,
    seed=42,
    validation_split=0.3,
    subset='validation')

Transfer learning

In [ ]:
base_model = keras.applications.MobileNetV3Large(
    weights='imagenet',
    input_shape=(150, 150, 3),
    include_top=False
)

base_model.trainable = False

# functional
inputs = keras.Input(shape=(150, 150, 3))
x = keras.applications.mobilenet.preprocess_input(inputs)
x = base_model(inputs, training=False)
x = keras.layers.Dropout(0.2)(x)
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

# sequential
model = keras.Sequential()
model.add(keras.layers.InputLayer(input_shape=(150, 150, 3)))
model.add(keras.applications.mobilenet.preprocess_input())
model.add(base_model(training=False))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.compile(
    optimizer=keras.optimizers.SGD(),
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy()]
)

model.fit(train_ds, epochs=10, validation_data=val_ds)

Fine tuning

In [ ]:
base_model.trainable = True

model.compile(
    optimizer=keras.optimizers.Adam(1e-5),  # Low learning rate
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy()],
)

epochs = 10
model.fit(train_ds, epochs=epochs, validation_data=val_ds)